# Image Tutorial
The current authoritative version of the full image XDS 
schema can be found at https://docs.google.com/spreadsheets/d/1WW0Gl6z85cJVPgtdgW4dxucurHFa06OKGjgoK8OREFA/edit#gid=1719181934

**Install packages**

In [1]:
import xradio
print(xradio.__file__)

/home/dmehring/projects/xradio/xradio/src/xradio/__init__.py


In [2]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.41 already installed.


In [3]:
from toolviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2025-02-14 16:44:41,742]     INFO    viperlog:  Checking functions availability: 
[2025-02-14 16:44:41,743]     INFO    viperlog:  Loading module: slurm -- Success 
[2025-02-14 16:44:41,743]     INFO    viperlog:     dask_jobqueue is available 
[2025-02-14 16:44:41,745]     INFO    viperlog:  Loading module: dask_ssh -- Fail 
[2025-02-14 16:44:41,745]     INFO    viperlog:     asyncssh is unavailable 
[2025-02-14 16:44:41,746]     INFO    viperlog:     jupyter_server_proxy is unavailable 
[2025-02-14 16:44:41,747]     INFO    viperlog:     paramiko is unavailable 
[2025-02-14 16:44:41,747]     INFO    viperlog:  Loading module: CUDA -- Fail 
[2025-02-14 16:44:41,748]     INFO    viperlog:     dask_cuda is unavailable 
[2025-02-14 16:44:41,748]     INFO    viperlog:  Available functions of this environment: slurm 
[2025-02-14 16:44:41,749]     INFO    viperlog:  Module path: /home/dmehring/miniforge3/envs/xradio/lib/python3.12/site-packages/toolviper 
[2025-02-14 16:44:41,758]  WARNING

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41381,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:33951,Total threads: 1
Dashboard: http://127.0.0.1:34549/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:44637,


**Imports**

In [4]:
import xradio

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

## Download image to be used

In [5]:
import toolviper
toolviper.utils.data.download("demo_simulated.im")

[2025-02-14 16:44:43,162]  WARNING    viperlog:  File exists: /home/dmehring/miniforge3/envs/xradio/lib/python3.12/site-packages/toolviper/utils/data/.dropbox 
[2025-02-14 16:44:43,163]     INFO    viperlog:  Updating file metadata information ...  
 

 Download List      
 ─────────────────── 
  demo_simulated.im

[2025-02-14 16:44:44,162]     INFO    viperlog:  File exists: demo_simulated.im 


## Convert CASA image to XDS

### Supported images, casa, FITS, zarr
Supported image formats that can be read
by this method are casacore, FITS, and zarr. Here "supported" means
some images in these formats are supported. The parameter space of
input image formats that has been investigated is still limited. For
example, aperture (*uv* plane) images are not yet supported, but given
their importance in applications like *feather* which make heavy use
of fourier transforms, support for them will likely be added sooner
rather than later. Note that images containing Taylor terms will also
be supported at some point.

### Always 5-D, time, polarization, frequency, l/u, m/v dimensions
Valid image XDSes always have five dimensions: time, frequency, polarization,
l (or u),
and m (or v). Images that have multiple (per-plane) beams will also have
a beam_param dimension. The dimensions will follow this ordering
in most, and maybe all, cases. That's still TBD.

###  Coordinates vs dimensions
Note that in xarray jargon, a dimension and a coordinate are
different constructs. Coordinates are "higher level" in that they are
always functions of dimensions. I think of dimensions as orthogonal
axes in a Cartesian coordinate system. They are essentially pixel coordinate
axes. Coordinates, however, do not
have to conform to this constraint, which is important for curved
surfaces like the sky. Think of RA and Dec, which are functions
of the two direction pixel coordinates, (l, m). So, xarray coordinates
are akin to CASA's world coordinates.

### read_image, create image xarray dataset (XDS)
The **read_image** function will convert an image in a supported format
to an image XDS (xarray.Dataset).

###  Output dask array chunksize specified as a dict to read_image()
The chunk size of the output dask pixel array can be specified using
a dictionary in which the supported keys are 'l' (corresponding to
the longitude like dimension), 'm' (corresponding to the latitude like
dimension), 'frequency' (corresponding to the spectral dimension), 'polarization'
(corresponding to the polarization dimension), and 'time'
(corresponding to the temporal dimension). The values correspond to
the chunk length along that axis. If any of these keys are absent, the
length of a chunk in that dimension will be equal to the number of
pixels along that axis. Specified keys for which there are no
corresponding axes in the CASA image are silently ignored. 'u' and 'v'
may be substituted for 'l' and 'm' which is useful in the case of
aperture (Fourier plane) images, when support for those has been added.

In [6]:
# polarization is not specified, so the length of the axis in this dimension
# is equal to the number of pixels on the polarization axis, which is 4.
imname = "demo_simulated.im"

xds = read_image(infile=imname, chunks={"l": 40, "m": 20, "frequency": 5})
xds

Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows
[2025-02-14 16:44:44,594]     INFO    viperlog:  J2000 found as native reference frame in CASA image This corresponds to fk5(equinox="j2000") in astropy. Metadata will be written appropriately 


<xarray.Dataset> Size: 20MB
Dimensions:          (time: 1, polarization: 4, frequency: 50, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity         (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 60B 'major' 'minor' 'pa'
Data variables:
    SKY              (time, polarization, frequency, l, m) float32 16MB dask.array<chunksize=(1, 4, 5, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool 4MB dask.array<chunksize=(1, 4, 5, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 5kB dask.array<chunksize=(1, 4, 50, 3), meta=np.ndarray>
Attributes:
    direction:        {'reference': {'attrs': {'frame': 'fk5', 'type': 'sky_c...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'elli...
    obsdate:          {'type': 'time', 'scale': 'UTC', 'units': 'd', 'value':...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      
    user:             {}
    active_mask:      mask0
    description:      None
    history:          <xarray.Dataset> Size: 11kB\nDimensions:    (row: 15)\n...

##  Data Variables

### Data Variable: Pixel Values

The image pixel values are contained in the sky data variable.

In [7]:
xds.SKY

<xarray.DataArray 'SKY' (time: 1, polarization: 4, frequency: 50, l: 200, m: 100)> Size: 16MB
dask.array<transpose, shape=(1, 4, 50, 200, 100), dtype=float32, chunksize=(1, 4, 5, 40, 20), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity         (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
Attributes:
    image_type:  Intensity
    units:       Jy/beam

###  Data Variable: Masks

**WARNING: 
The meaning of xradio image mask values follows the numpy masked array
convention which is opposite to CASA, ie True means the pixel is good, 
False means it is bad**

The active mask can be found in the xds 'active_mask' attr (more on attributes later). An empty string indicates that no mask is active; that is, all pixels are considered "good". Our example
image does have a mask, and it is stored in the mask0 data variable.

In [8]:
active_mask = xds.attrs["active_mask"]
active_mask

'mask0'

Mask values are stored in the xds data variable named for the mask

In [9]:
xds[active_mask]

<xarray.DataArray 'mask0' (time: 1, polarization: 4, frequency: 50, l: 200,
                           m: 100)> Size: 4MB
dask.array<logical_not, shape=(1, 4, 50, 200, 100), dtype=bool, chunksize=(1, 4, 5, 40, 20), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity         (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
Attributes:
    image_type:  Mask

Or, via XDS 'dot' notation

In [10]:
xds.mask0

<xarray.DataArray 'mask0' (time: 1, polarization: 4, frequency: 50, l: 200,
                           m: 100)> Size: 4MB
dask.array<logical_not, shape=(1, 4, 50, 200, 100), dtype=bool, chunksize=(1, 4, 5, 40, 20), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity         (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
Attributes:
    image_type:  Mask

###  Data Variable: Multiple Beams

For a multi (per-plane) beam image, beams are stored in the 'beam' data variable so an XDS selection results in the correct beams being selected. If the image does not have multiple beams, there is instead an xds.beam attribute. If the image has a single beam, it is stored there in a CASA-like beam dictionary {major, minor, pa}. If the image has no beam then xds.attrs['beam']=None. The image in this demo has multiple beams.

In [11]:
xds.beam

<xarray.DataArray 'beam' (time: 1, polarization: 4, frequency: 50, beam_param: 3)> Size: 5kB
dask.array<array, shape=(1, 4, 50, 3), dtype=float64, chunksize=(1, 4, 50, 3), chunktype=numpy.ndarray>
Coordinates:
  * time          (time) float64 8B 5.154e+04
  * polarization  (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency     (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity      (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * beam_param    (beam_param) <U5 60B 'major' 'minor' 'pa'
Attributes:
    units:    rad

All beam parameters are stored in a single array. The final dimension in the beam array is 'beam_param'. The length of this axis is always three. The three "coordinate" labels for that axis are for major axis, minor axis, and position angle. The values on the beam_param axis are labeled to allow easy selection using xarray selection methods. For example, one can easily select only the major axis values. Normally, we don't want to drop degenerate dimensions produced by selections, so to avoid that we put the value being selected in square brackets (more on selecting data later).

In [12]:
xds.beam.sel(beam_param=["major"])

<xarray.DataArray 'beam' (time: 1, polarization: 4, frequency: 50, beam_param: 1)> Size: 2kB
dask.array<getitem, shape=(1, 4, 50, 1), dtype=float64, chunksize=(1, 4, 50, 1), chunktype=numpy.ndarray>
Coordinates:
  * time          (time) float64 8B 5.154e+04
  * polarization  (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency     (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity      (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * beam_param    (beam_param) <U5 20B 'major'
Attributes:
    units:    rad

## Coordinates

### Accessing Coordinates

Coordinate are stored as xarray DataArrays. A coordinate may have
attributes that provide metadata pertaining to the coordinate.

You can explicitly indicate a coordinate is a by referring to the coords
dict of the xds. This may make your code more readable. However you can
also refer to them using the 'dot' notation, and because a coordinate
sometimes has the same name as a dimension (and so is called a dimension
coordinate), and because when indexing dimensions you always use the
'dot' notation, using the 'dot' notation for coordinates as well is
probably ok from a best practice point of view. So, my opinion is
either-or.

In [13]:
xds.coords["time"]
# or
xds.time
# or
xds["time"]

<xarray.DataArray 'time' (time: 1)> Size: 8B
array([51544.])
Coordinates:
  * time     (time) float64 8B 5.154e+04
Attributes:
    type:     time
    scale:    UTC
    units:    d
    format:   MJD

In [14]:
xds.polarization

<xarray.DataArray 'polarization' (polarization: 4)> Size: 16B
array(['I', 'Q', 'U', 'V'], dtype='<U1')
Coordinates:
  * polarization  (polarization) <U1 16B 'I' 'Q' 'U' 'V'

In [15]:
xds.frequency

<xarray.DataArray 'frequency' (frequency: 50)> Size: 400B
array([1.414975e+09, 1.414976e+09, 1.414977e+09, 1.414978e+09, 1.414979e+09,
       1.414980e+09, 1.414981e+09, 1.414982e+09, 1.414983e+09, 1.414984e+09,
       1.414985e+09, 1.414986e+09, 1.414987e+09, 1.414988e+09, 1.414989e+09,
       1.414990e+09, 1.414991e+09, 1.414992e+09, 1.414993e+09, 1.414994e+09,
       1.414995e+09, 1.414996e+09, 1.414997e+09, 1.414998e+09, 1.414999e+09,
       1.415000e+09, 1.415001e+09, 1.415002e+09, 1.415003e+09, 1.415004e+09,
       1.415005e+09, 1.415006e+09, 1.415007e+09, 1.415008e+09, 1.415009e+09,
       1.415010e+09, 1.415011e+09, 1.415012e+09, 1.415013e+09, 1.415014e+09,
       1.415015e+09, 1.415016e+09, 1.415017e+09, 1.415018e+09, 1.415019e+09,
       1.415020e+09, 1.415021e+09, 1.415022e+09, 1.415023e+09, 1.415024e+09])
Coordinates:
  * frequency  (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity   (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
Attributes:
    rest_frequency:   {'data': 1420405751.7860003, 'dims': [], 'attrs': {'uni...
    wave_unit:        mm
    reference_value:  {'attrs': {'units': ['Hz'], 'observer': 'lsrk', 'type':...

In [16]:
xds.velocity

<xarray.DataArray 'velocity' (frequency: 50)> Size: 400B
array([1146220.665937  , 1146009.60479647, 1145798.54365594,
       1145587.48251538, 1145376.42137485, 1145165.36023431,
       1144954.29909378, 1144743.23795325, 1144532.17681269,
       1144321.11567215, 1144110.05453162, 1143898.99339109,
       1143687.93225053, 1143476.87111   , 1143265.80996946,
       1143054.74882893, 1142843.6876884 , 1142632.62654784,
       1142421.56540731, 1142210.50426677, 1141999.44312624,
       1141788.38198568, 1141577.32084515, 1141366.25970461,
       1141155.19856408, 1140944.13742352, 1140733.07628299,
       1140522.01514246, 1140310.95400192, 1140099.89286139,
       1139888.83172083, 1139677.7705803 , 1139466.70943977,
       1139255.64829923, 1139044.58715867, 1138833.52601814,
       1138622.46487761, 1138411.40373707, 1138200.34259654,
       1137989.28145598, 1137778.22031545, 1137567.15917492,
       1137356.09803438, 1137145.03689382, 1136933.97575329,
       1136722.91461276, 1136511.85347223, 1136300.79233169,
       1136089.73119113, 1135878.6700506 ])
Coordinates:
  * frequency  (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity   (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
Attributes:
    doppler_type:  radio
    units:         ['m/s']
    type:          doppler

For astronomical images, the longitude-like (eg, RA) and latitude-like
(eg, Dec) coordinates are functions of both the *l* and *m* dimensions
and so are 2-D arrays to properly account for sky curvature. These
values are generally in radians. The coordinate name will depend on what
is represented, eg it could be galactic_longitude for an image that uses
galactic coordinates.

In [17]:
xds.right_ascension

<xarray.DataArray 'right_ascension' (l: 200, m: 100)> Size: 160kB
array([[0.029096  , 0.02909588, 0.02909576, ..., 0.02909564, 0.02909576,
        0.02909588],
       [0.02880496, 0.02880484, 0.02880472, ..., 0.02880461, 0.02880472,
        0.02880484],
       [0.02851392, 0.0285138 , 0.02851369, ..., 0.02851357, 0.02851369,
        0.0285138 ],
       ...,
       [6.25496242, 6.25496254, 6.25496265, ..., 6.25496277, 6.25496265,
        6.25496254],
       [6.25467138, 6.2546715 , 6.25467162, ..., 6.25467173, 6.25467162,
        6.2546715 ],
       [6.25438034, 6.25438047, 6.25438058, ..., 6.2543807 , 6.25438058,
        6.25438047]])
Coordinates:
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425

In [18]:
xds.declination

<xarray.DataArray 'declination' (l: 200, m: 100)> Size: 160kB
array([[-0.01454492, -0.014254  , -0.01396309, ...,  0.01367217,
         0.01396309,  0.014254  ],
       [-0.01454492, -0.014254  , -0.01396309, ...,  0.01367217,
         0.01396309,  0.014254  ],
       [-0.01454492, -0.014254  , -0.01396309, ...,  0.01367217,
         0.01396309,  0.014254  ],
       ...,
       [-0.01454492, -0.014254  , -0.01396309, ...,  0.01367217,
         0.01396309,  0.014254  ],
       [-0.01454492, -0.014254  , -0.01396309, ...,  0.01367217,
         0.01396309,  0.014254  ],
       [-0.01454492, -0.014254  , -0.01396309, ...,  0.01367217,
         0.01396309,  0.014254  ]])
Coordinates:
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425

### Coordinate Attributes

All coordinates, except polarization, have attributes, which are metadata that describe the coordinate. These are dictionaries that are stored in the attrs field of the coordinate.

#### attrs is a dict and accessible via the attrs field

In [19]:
xds.time.attrs

{'type': 'time', 'scale': 'UTC', 'units': 'd', 'format': 'MJD'}

####  prefer to access attrs via the explicit attrs field rather than dot notation
You can again access attributes with the 'dot' notation, but because
attributes can be numerous, I generally indicate explicitly that I'm
referring to an attribute by explicitly naming the attrs field. IMO this
is best practice when referring to attributes. It's a bit easier to 
distinguish coordinate attributes using the 'dot' notation than it is
to distinguish dataset attributes (discussed later), but for
consistency I like to explicitly refer to the attrs field in both cases.

In [20]:
# So I don't recommend this, even though it is valid
xds.time.units
# I instead recommend this
xds.time.attrs["units"]

'd'

In [21]:
xds.frequency.attrs

{'rest_frequency': {'data': 1420405751.7860003,
  'dims': [],
  'attrs': {'units': ['Hz'], 'type': 'quantity'}},
 'wave_unit': 'mm',
 'reference_value': {'attrs': {'units': ['Hz'],
   'observer': 'lsrk',
   'type': 'frequency'},
  'data': 1415000000.0,
  'dims': []}}

In [22]:
xds.velocity.attrs

{'doppler_type': 'radio', 'units': ['m/s'], 'type': 'doppler'}

For astronomical images, the longitude-like and latitude-like coordinate have both seperable attributes and shared attributes. The separable attributes are stored with the coordinate. The shared attributes are stored in the 'direction' attribute of the xds.

In [23]:
xds.right_ascension.attrs

{}

In [24]:
xds.declination.attrs

{}

##  Dateset Level Attributes

Metadata that apply to the dataset as a whole are stored in dataset
level attributes. Most of these come right from the CASA image
metadata.

In [25]:
xds.attrs.keys()

dict_keys(['direction', 'telescope', 'obsdate', 'pointing_center', 'observer', 'object_name', 'user', 'active_mask', 'description', 'history'])

This is the shared direction coordinate metadata at the xds level. Note that system='J2000' in the CASA image maps to system='FK5' (the frame in astropy speak) and equinox='J2000' in the xds image. This is how astropy requires these details to be provided.

In [26]:
xds.attrs["direction"]

{'reference': {'attrs': {'frame': 'fk5',
   'type': 'sky_coord',
   'units': ['rad', 'rad'],
   'equinox': 'j2000.0'},
  'data': [np.float64(0.0), np.float64(0.0)],
  'dims': ['l', 'm']},
 'latpole': {'data': 0.0,
  'dims': ['l', 'm'],
  'attrs': {'units': ['rad'], 'type': 'quantity'}},
 'lonpole': {'data': 3.141592653589793,
  'dims': ['l', 'm'],
  'attrs': {'units': ['rad'], 'type': 'quantity'}},
 'pc': array([[1., 0.],
        [0., 1.]]),
 'projection_parameters': array([0., 0.]),
 'projection': 'SIN'}

In [27]:
xds.attrs["telescope"]

{'name': 'ALMA',
 'position': {'type': 'position',
  'ellipsoid': 'GRS80',
  'units': ['rad', 'rad', 'm'],
  'value': [-1.1825465955049892, -0.3994149869262738, 6379946.01326443]}}

In [28]:
xds.attrs["observer"]

'Karl Jansky'

In [29]:
xds.attrs["obsdate"]

{'type': 'time',
 'scale': 'UTC',
 'units': 'd',
 'value': 51544.00000000116,
 'format': 'MJD'}

In [30]:
xds.attrs["pointing_center"]

{'value': array([0., 0.]), 'initial': True}

In [31]:
xds.attrs["object_name"]

''

### Cannot set attr using dot notation
xarray does not permit setting of attributes using 'dot' notation.

In [32]:
try:
    xds.object_name = "SgrB2"
    print(
        "Wow! You have superhuman python powers. We would like to hire you as an "
        "assistant superhero python developer with the Avengers!"
    )
except Exception as e:
    print(e)
    print(
        "\nAww. You only have regular human python powers. Thanks for applying for a "
        "position with the Avengers, but we have no use for your feeble regular human "
        "python powers at this time."
    )

cannot set attribute 'object_name' on a 'Dataset' object. Use __setitem__ styleassignment (e.g., `ds['name'] = ...`) instead of assigning variables.

Aww. You only have regular human python powers. Thanks for applying for a position with the Avengers, but we have no use for your feeble regular human python powers at this time.


In [33]:
xds.attrs["object_name"] = "SgrB2"
xds.attrs["object_name"]

'SgrB2'

### User attr dictionary
The 'user' attribute is for user specified metadata. This is also where metadata from the CASA image 'miscinfo' dictionary are written.

In [34]:
xds.attrs["user"]["phone"] = 123456
xds.attrs["user"]

{'phone': 123456}

In [35]:
xds.attrs["description"]  # this image has no description

In [36]:
xds.attrs["description"] = "my way cool pretend image"
xds.attrs["description"]

'my way cool pretend image'

The history is currently stored as an xarray Dataset but the specific
format will likely change once a format that can be used for both
images and MSes has been specified

In [37]:
xds.attrs["history"]

<xarray.Dataset> Size: 11kB
Dimensions:    (row: 15)
Coordinates:
    OBJECT_ID  (row) <U1 60B '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
Dimensions without coordinates: row
Data variables:
    TIME       (row) float64 120B 5.19e+09 5.19e+09 ... 5.19e+09 5.19e+09
    PRIORITY   (row) <U4 240B 'INFO' 'INFO' 'INFO' ... 'INFO' 'INFO' 'INFO'
    MESSAGE    (row) <U150 9kB "Created Paged image 'demo.im' of shape [200, ...
    LOCATION   (row) <U27 2kB '::ImageFactory::createImage' ... '::image::put...
Attributes:
    column_descriptions:  {'TIME': {'valueType': 'double', 'dataManagerType':...
    info:                 {'type': 'Log message', 'subType': '', 'readme': 'R...
    bad_cols:             []

In [38]:
xds.attrs["history"].TIME

<xarray.DataArray 'TIME' (row: 15)> Size: 120B
array([5.18959414e+09, 5.18959414e+09, 5.18959414e+09, 5.18959416e+09,
       5.18959416e+09, 5.18959521e+09, 5.18959521e+09, 5.18961376e+09,
       5.18961376e+09, 5.18961377e+09, 5.18961377e+09, 5.18993385e+09,
       5.18993385e+09, 5.18996290e+09, 5.18996290e+09])
Coordinates:
    OBJECT_ID  (row) <U1 60B '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
Dimensions without coordinates: row

In [39]:
xds.attrs["history"].MESSAGE

<xarray.DataArray 'MESSAGE' (row: 15)> Size: 9kB
array(["Created Paged image 'demo.im' of shape [200, 100, 50, 4] with float valued pixels.",
       'Ran ia.fromshape',
       'ia.fromshape(outfile="demo.im", shape=[200,100,50,4], csys={...}, linear=false, overwrite=false, log=true, type="f")',
       'Ran ia.addnoise',
       'ia.addnoise(type="normal", pars=[0.000000,1.000000], region={...}, zeroit=false, seeds=[])',
       'Ran ia.subimage',
       'ia.subimage(outfile="demo1.im", region="", mask="demo.im > 0", dropdeg=false, overwrite=false, list=true, stretch=false, wantreturn=true, keepaxes=[])',
       'Ran ia.setrestoringbeam',
       'ia.setrestoringbeam(major="5arcmin", minor="4arcmin", pa="20deg", beam={...}, remove=false, log=true, channel=0, polarization=0, imagename="")',
       'Ran ia.setrestoringbeam',
       'ia.setrestoringbeam(major="5arcmin", minor="4arcmin", pa="10deg", beam={...}, remove=false, log=true, channel=1, polarization=1, imagename="")',
       'Ran ia.setbrightnessunit', 'ia.setbrightnessunit(unit="Jy/beam")',
       'Ran ia.putchunk',
       'ia.putchunk(pixels="[...]", blc=[-1], inc=[1], list=false, locking=true, replicate=false)'],
      dtype='<U150')
Coordinates:
    OBJECT_ID  (row) <U1 60B '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
Dimensions without coordinates: row

##  Bonus Section: Measures in xradio
xradio will support a few types of measures (and the image schema will be
updated to support those very soon). A measure will have a type key indicating
what type of measure it is. There will be a quantity type

In [40]:
{"data": 4.0, "dims": [], "attrs": {"type": "quantity", "units": ["m"]}}

{'data': 4.0, 'dims': [], 'attrs': {'type': 'quantity', 'units': ['m']}}

time type

In [41]:
{"data": 1697666092, "dims": [], "attrs": {"type": "time", "units": ["s"], "scale": "utc", "format": "unix"}}

{'data': 1697666092,
 'dims': [],
 'attrs': {'type': 'time', 'units': ['s'], 'scale': 'utc', 'format': 'unix'}}

spectral_coord type

In [42]:
{
    "data": 1.415e9,
    "dims": [],
    "attrs": {
        "type": "spectral_coord",
        "units": ["Hz"],
        "observer": "lsrk"
    }
}

{'data': 1415000000.0,
 'dims': [],
 'attrs': {'type': 'spectral_coord', 'units': ['Hz'], 'observer': 'lsrk'}}

sky_coord type

In [43]:
{"data": [0.5, -1.0], "dims": ["l", "m"], "attrs": {"type": "sky_coord", "units": ["rad", "rad"], "frame": "fk5"}}

{'data': [0.5, -1.0],
 'dims': ['l', 'm'],
 'attrs': {'type': 'sky_coord', 'units': ['rad', 'rad'], 'frame': 'fk5'}}

(geo) location type

In [44]:
{
    "data": [4e6, 3e6, 5e6],
    "attrs": {
        "type": "earch_location",
        "frame": "ITRF",
        "coordinate_system": "geocentric",
        "units": ["m", "m", "m"],
        "origin_object_name": "earth"
    }
}

{'data': [4000000.0, 3000000.0, 5000000.0],
 'attrs': {'type': 'earch_location',
  'frame': 'ITRF',
  'coordinate_system': 'geocentric',
  'units': ['m', 'm', 'm'],
  'origin_object_name': 'earth'}}

and doppler type

In [45]:
{
    "data": 60000,
    "dims": ["frequency"],
    "attrs": {
        "type": "doppler",
        "units": ["m/s"],
        "doppler_type": "radio"
    }
}

{'data': 60000,
 'dims': ['frequency'],
 'attrs': {'type': 'doppler', 'units': ['m/s'], 'doppler_type': 'radio'}}

##  Data Selection

###  Using numpy indexing (but don't do this)

Basic numpy indexing and slicing can be used to get values, although this method is the least recommended since it assumes the axis ordering. Better ways are discussed later.

####  values returns a numpy array
'values' returns a numpy array. Note that with the specified selection below, the output polarization axis will be degenerate. 

#### Carry along all dimensions when selecting to end up with a valid image XDS
The degenerate axis is kept by specifying the corresponding axis selection as [0], rather than 0. In the latter case, the axis will be dropped. In general, we want to carry all dimensions along; an image XDS pixel and mask data variables should always be five dimensional to avoid any ambiguity of dimension number to coordinate mapping.

In [46]:
vals = xds.SKY[:, [0], 8:9, 5:7, 8:9].values
vals

array([[[[[ 0.39568448],
          [-0.94448763]]]]], dtype=float32)

####  compute() returns an xarray.DataArray
Calling compute() returns an xarray DataArray. Similar to above, if the
polarization axis selection is not enclosed in [], the pol dimension will
be dropped in the output DataArray.

In [47]:
xda = xds.SKY[:, [0], 8:9, 5:7, 8:9].compute()
xda

<xarray.DataArray 'SKY' (time: 1, polarization: 1, frequency: 1, l: 2, m: 1)> Size: 8B
array([[[[[ 0.39568448],
          [-0.94448763]]]]], dtype=float32)
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 4B 'I'
  * frequency        (frequency) float64 8B 1.415e+09
    velocity         (frequency) float64 8B 1.145e+06
  * l                (l) float64 16B 0.02763 0.02734
  * m                (m) float64 8B -0.01222
    right_ascension  (l, m) float64 16B 0.02764 0.02735
    declination      (l, m) float64 16B -0.01222 -0.01222
Attributes:
    image_type:  Intensity
    units:       Jy/beam

#### But don't use numpy selection syntax, instead select by dimensions and/or coordinates
In general, numpy selection syntax is discouraged. It is preferred instead that selection make use of xarray style selection which is based on dimension and coordinate names and labels.

 ### Using `isel()`

One can select xarray data using indexing by dimension names.
This can be done at the xarray.DataArray level, or the xarray.Dataset level.

For example, to select stokes I from the first two frequency planes in the Dataset using integer array indices. Remember the numpy selection syntax used to preserve dimensionality when there is a degenerate axis in the output? That applies here as well

In [48]:
freq_0_1 = xds.isel({"frequency": slice(0, 2), "polarization": [0]})
freq_0_1

<xarray.Dataset> Size: 523kB
Dimensions:          (time: 1, polarization: 1, frequency: 2, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 4B 'I'
  * frequency        (frequency) float64 16B 1.415e+09 1.415e+09
    velocity         (frequency) float64 16B 1.146e+06 1.146e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 60B 'major' 'minor' 'pa'
Data variables:
    SKY              (time, polarization, frequency, l, m) float32 160kB dask.array<chunksize=(1, 1, 2, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool 40kB dask.array<chunksize=(1, 1, 2, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 48B dask.array<chunksize=(1, 1, 2, 3), meta=np.ndarray>
Attributes:
    direction:        {'reference': {'attrs': {'frame': 'fk5', 'type': 'sky_c...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'elli...
    obsdate:          {'type': 'time', 'scale': 'UTC', 'units': 'd', 'value':...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      SgrB2
    user:             {'phone': 123456}
    active_mask:      mask0
    description:      my way cool pretend image
    history:          <xarray.Dataset> Size: 11kB\nDimensions:    (row: 15)\n...

###  Using `sel()`

Or select by coordinate labels using sel().

In [49]:
freq_0_1_labels = xds.sel(frequency=slice(1.414975e9, 1.414976e9), polarization=["I"])
freq_0_1_labels

<xarray.Dataset> Size: 523kB
Dimensions:          (time: 1, polarization: 1, frequency: 2, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 4B 'I'
  * frequency        (frequency) float64 16B 1.415e+09 1.415e+09
    velocity         (frequency) float64 16B 1.146e+06 1.146e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 60B 'major' 'minor' 'pa'
Data variables:
    SKY              (time, polarization, frequency, l, m) float32 160kB dask.array<chunksize=(1, 1, 2, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool 40kB dask.array<chunksize=(1, 1, 2, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 48B dask.array<chunksize=(1, 1, 2, 3), meta=np.ndarray>
Attributes:
    direction:        {'reference': {'attrs': {'frame': 'fk5', 'type': 'sky_c...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'elli...
    obsdate:          {'type': 'time', 'scale': 'UTC', 'units': 'd', 'value':...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      SgrB2
    user:             {'phone': 123456}
    active_mask:      mask0
    description:      my way cool pretend image
    history:          <xarray.Dataset> Size: 11kB\nDimensions:    (row: 15)\n...

In [50]:
(freq_0_1.SKY.values == freq_0_1_labels.SKY.values).all()

np.True_

###  Chaining `isel()` and `sel()`

sel() and isel() can be chained for maximum flexibility. Don't forget to express the pol selection as a list so dimensions are not dropped

In [51]:
freq_0_1_isel_sel = xds.isel(frequency=slice(0, 2)).sel(polarization=["I"])
freq_0_1_isel_sel

<xarray.Dataset> Size: 523kB
Dimensions:          (time: 1, polarization: 1, frequency: 2, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 4B 'I'
  * frequency        (frequency) float64 16B 1.415e+09 1.415e+09
    velocity         (frequency) float64 16B 1.146e+06 1.146e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 60B 'major' 'minor' 'pa'
Data variables:
    SKY              (time, polarization, frequency, l, m) float32 160kB dask.array<chunksize=(1, 1, 2, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool 40kB dask.array<chunksize=(1, 1, 2, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 48B dask.array<chunksize=(1, 1, 2, 3), meta=np.ndarray>
Attributes:
    direction:        {'reference': {'attrs': {'frame': 'fk5', 'type': 'sky_c...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'elli...
    obsdate:          {'type': 'time', 'scale': 'UTC', 'units': 'd', 'value':...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      SgrB2
    user:             {'phone': 123456}
    active_mask:      mask0
    description:      my way cool pretend image
    history:          <xarray.Dataset> Size: 11kB\nDimensions:    (row: 15)\n...

In [52]:
(freq_0_1.SKY.values == freq_0_1_isel_sel.SKY.values).all()

np.True_

### Using where() to define image regions
where() is useful for ~~selecting~~ deliniating regions via masking. where() does not select regions, and so it is fundamentally different from isel() and sel() in this regard. It instead conforms to the rules of binary math operations on xarray.Datasets (https://docs.xarray.dev/en/stable/user-guide/computation.html).

This is important to understand for what we are about to do next. We are going to mask a region based on a condition specified by what can be thought of as a general function of l and m. This will do what you might expect for xds.sky, but for xds.beam, it will add dimensions l and m because of broadcasting rules. So, here we really just want to mask xds.sky and so will run where() on the xds.sky DataArray, not the entire xds Dataset.

Here we select a circular region of radius=40 pixels centered at the image center. Pixel values outside the region are masked (set to nan). We first make a copy so we do not alter the xds dataset.

In [53]:
circle_region = xds.copy(deep=True)

In [54]:
# Note that this code assumes l and m are dimensions, not
# dimension coordinates. When there are also l and m
# dimension coordinates, the following transformations
# will produce the same result as below
# xds.l -> xds.dims['l'], xds.m -> xds.dims['m']
# ie when there is a dimension and dimension coordinate, just
# referring to the dimension/dimension coordinate name means
# where() will use the coordinate values. If you want to refer
# to the dimension, you explicitly have to express that via
# the dims['x'] field.
l_shift = xds.l - len(xds.l) / 2
m_shift = xds.m - len(xds.m) / 2
# deliniate a circular region of radius 40 pixels in
# circle_region.sky only
circle_region["sky"] = circle_region.SKY.where(l_shift**2 + m_shift**2 <= 1600)
circle_region

<xarray.Dataset> Size: 36MB
Dimensions:          (time: 1, polarization: 4, frequency: 50, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 8B 5.154e+04
  * polarization     (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 400B 1.415e+09 1.415e+09 ... 1.415e+09
    velocity         (frequency) float64 400B 1.146e+06 1.146e+06 ... 1.136e+06
  * l                (l) float64 2kB 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 800B -0.01454 -0.01425 ... 0.01396 0.01425
    right_ascension  (l, m) float64 160kB 0.0291 0.0291 0.0291 ... 6.254 6.254
    declination      (l, m) float64 160kB -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 60B 'major' 'minor' 'pa'
Data variables:
    SKY              (time, polarization, frequency, l, m) float32 16MB dask.array<chunksize=(1, 4, 5, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool 4MB dask.array<chunksize=(1, 4, 5, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 5kB dask.array<chunksize=(1, 4, 50, 3), meta=np.ndarray>
    sky              (time, polarization, frequency, l, m) float32 16MB dask.array<chunksize=(1, 4, 5, 40, 20), meta=np.ndarray>
Attributes:
    direction:        {'reference': {'attrs': {'frame': 'fk5', 'type': 'sky_c...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'elli...
    obsdate:          {'type': 'time', 'scale': 'UTC', 'units': 'd', 'value':...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      SgrB2
    user:             {'phone': 123456}
    active_mask:      mask0
    description:      my way cool pretend image
    history:          <xarray.Dataset> Size: 11kB\nDimensions:    (row: 15)\n...

###  Example of Complex Region Selection

Plot the central region of polarizaton=I and the first frequency channel.

In [ ]:
# before region selection, note that we are not using the image pixel mask,
# mask0, for plotting here. This is just a quick and dirty demo; actual
# xradio/ngCASA applications will use pixel masks as expected.
data_sel = {"time": [0], "polarization": [0], "frequency": [0], "l": slice(50, 150)}
clim = {"vmin": -2, "vmax": 6}
xds.SKY.isel(data_sel).plot(**clim)

Deliniate a circular region of radius 40 pixels. Pixel values in the region outside the center circle are NaN.

In [ ]:
circular_region = xds.copy(deep=True)
l_shift = xds.l - xds.l[len(xds.l) // 2]
m_shift = xds.m - xds.m[len(xds.m) // 2]
radius = 40 * abs(xds.l.cdelt)
circular_region["SKY"] = circular_region["SKY"].where(
    l_shift**2 + m_shift**2 <= radius * radius
)
circular_region

In [ ]:
# Here is what the circular region deliniation looks like. Pixel values
# outside the selected region are NaN.
circular_region.SKY.isel(data_sel).plot(**clim)

In [ ]:
import numpy as np

np.isnan(circular_region.SKY.values).all()

### Pac Man!
With just a little more effort, a Pac Man region can be created.

In [ ]:
import numpy as np

# just rename the object, no copying is done here
pacman = circular_region
pacman["SKY"] = circular_region.SKY.where(
    np.logical_or(
        np.logical_or(m_shift < 0, l_shift - m_shift > 0), l_shift + m_shift < 0
    )
)
pacman.SKY.isel(data_sel).plot(**clim)

Below in the section on converting an image xds to a CASA image, we will write the pacman xds
to a CASA image and demonstrate that the CASA image pixel values agree with the pacman.sky
array values.

##  Loading Image Data Into Memory
In order to process an image's pixel data, it must be loaded into memory. This data must be immediately loaded, so that it cannot be a Dask array. The general architecture is that the loaded data will be placed in a dask.delayed function for processing.

A loaded data is itself a valid xds image (it can be correctly thought of as a subimage).

In [ ]:
subim = load_image(
    "demo_simulated.im",
    block_des={
        "time": 0,
        "polarization": 0,
        "frequency": slice(0, 10),
        "l": slice(40, 80),
        "m": slice(10, 20),
    },
    do_sky_coords=True,
)
subim

### Loaded data is a numpy array

In [ ]:
subim.SKY

In [ ]:
subim.mask0

A numpy.ma.masked_array can be created from the pixels and the mask

In [ ]:
import numpy as np

ma = np.ma.masked_array(subim.SKY[:], subim.mask0[:])
ma

### Loaded data can be used in a dask.delayed function
Now the masked array can be put in a dask.delayed function, and things can be done with it.

In [ ]:
import dask.delayed


def summer(ary):
    return ary.sum()


summer = dask.delayed(summer)(ma)
print(ma.shape)
print("summer type", type(summer))

In [ ]:
mysum = summer.compute()
mysum

## Convert an xds image to a CASA image

In [ ]:
# For this example, arrays are small so force arrays into a single chunk
# so writing doesn't take forever
xds = xds.chunk("auto")
write_image(xds, "new_casa.im", out_format="casa")

In [ ]:
from casacore.images import image as cimage

myim = cimage("new_casa.im")
ia_orig = cimage(imname)

### Axes order may be different (pol,freq <-> freq, pol), so transpose as necessary
The original image has axes in RA, Dec, Freq, Stokes order. Conversion of an xds image XDS
to a casa image results in a casacore image with axes in the order tclean currently writes,
RA, Dec, Stokes, Freq, so we must transpose the pixel values of one to compare to the pixel
values of the other. And the python-casacore image methods usually access pixels in the
reverse order you would expect.

In [ ]:
(myim.getdata() == ia_orig.getdata().transpose(1, 0, 2, 3)).all()

In [ ]:
got = myim.statistics()
del got["minpos"], got["maxpos"]
exp = ia_orig.statistics()
del exp["minpos"], exp["maxpos"]
for k in exp.keys():
    print(k, np.isclose(got[k][0], exp[k][0]))

In [ ]:
(myim.getmask() == ia_orig.getmask().transpose(1, 0, 2, 3)).all()

In [ ]:
# There is no obvious way to close() an image using
# the python-casacore API, so delete the objects so
# they are removed from the CASA table cache
del myim
del ia_orig

### Write pacman to CASA image

Export the pacman image xds to casa image and plot those pixel data
to verify the sky data and casa image data are the same

In [ ]:
# try/except needed for debugging in case the cell needs
# to be run multiple times, the CASA image must be deleted
# to remove it from the CASA table cache. Just do nothing
# if the CASA image hasn't been created yet.
try:
    del pacman_casa
except NameError as e:
    pass

pacman = pacman.chunk("auto")
write_image(pacman, "pacman.im", "casa")
from casacore.images import image as cimage

pacman_casa = cimage("pacman.im")
pacman_casa_data = pacman_casa.getdata()
import xarray as xr

xr_pacman_casa_data = xr.DataArray(
    pacman_casa_data, dims=["frequency", "polarization", "m", "l"]
)
casa_data_sel = data_sel.copy()
del casa_data_sel["time"]
# note that m and l are swapped relative to the pacman xds, so
# to easily plot  x-axis = l and y-axis = m, transpose
xr_pacman_casa_data = xr_pacman_casa_data.transpose(
    "frequency", "polarization", "l", "m"
)
xr_pacman_casa_data.isel(casa_data_sel).plot(**clim)
del pacman_casa